In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from math import sin, cos, pi

In [ ]:
verbose = True

In [ ]:
def rot(theta):
    return np.array([
        [cos(theta), -sin(theta)],
        [sin(theta), cos(theta)]
    ])


def to_deg(theta):
    return theta / pi * 180

In [ ]:

class Pose:
    def __init__(self, t:float, s:np.ndarray|list[float], theta:float):
        self.t = t
        self.s = np.array(s)
        self.theta = theta


    def advance(self, v:'Velocity', dt:float) -> 'Pose':
        return Pose(
            self.t + dt, 
            self.s + v.v * dt, 
            self.theta + v.omega * dt)
    
    
    def __str__(self):
        return f'pose ({self.s[0]:.1f}, {self.s[1]:.1f})cm, {to_deg(self.theta):.1f}° @ {self.t:.3f}s'
    

class Velocity:
    def __init__(self, v:np.ndarray|list[float], omega:float):
        self.v = np.array(v)
        self.omega = omega


    def advance(self, a:'Accel', dt:float) -> 'Velocity':
        return Velocity(
            self.v + a.a * dt,
            self.omega + a.alpha * dt)

    
    def __str__(self):
        return f'vel ({self.v[0]:.1f}, {self.v[1]:.1f})cm/s, {to_deg(self.omega):.1f}°/s'
    

class Accel:
    def __init__(self, a:np.ndarray|list[float], alpha:float):
        self.a = np.array(a)
        self.alpha = alpha

    
    def __str__(self):
        return f'acc ({self.a[0]:.1f}, {self.a[1]:.1f})cm/s^2, {to_deg(self.alpha):.1f}°/s^2'


In [ ]:
class SystemPose:
    def __init__(self, pose:Pose, rel_sw_pose:list[Pose]):
        self.pose = pose
        self.rel_sw_pose = rel_sw_pose


    def advance(self, vel:Velocity, sw_rel_vel:list[Velocity], dt:float):
        return SystemPose(
            self.pose.advance(vel, dt),
            [p.advance(v, dt) for (p, v) in zip(self.rel_sw_pose, sw_rel_vel)])
    

    def to_abs(self, i:int):
        return self.pose.s + rot(self.pose.theta) @ self.rel_sw_pose[i].s


    def __str__(self):
        s = str(self.pose)
        for sw in self.rel_sw_pose:
            s = f'{s} ({sw.theta:.1f}°)'
        
        return s

In [ ]:
# p0 = Pose(0, [2, 3], .4)
# p1 = Pose(0, [2.5, 4], -.3)
# print(p0.to_absolute(p1))

In [ ]:
class SwerveModule:
    def __init__(self, offset:np.ndarray|list[float], fstatic:np.ndarray|list[float], 
                 max_speed:float,
                 max_yaw_rate:float,
                 max_force:float):
        self.offset = np.array(offset)
        self.fstatic = np.array(fstatic)
        self.max_yaw_rate = max_yaw_rate
        self.max_force = max_force
        self.drag = np.array([max_force / max_speed, 0])


    def __str__(self):
        return f'swerve ({self.offset[0]:.1f}, {self.offset[1]:.1f})cm  ({self.fstatic[0]:.1f}, {self.fstatic[1]:.1f})kg.cm/s^2  ({self.drag[0]:.1f}, {self.drag[1]:.1f})kg/s'

In [ ]:
class System:
    def __init__(self, mass:float, moment:float, swerves:list[SwerveModule]):
        self.mass = mass
        self.moment = moment
        self.swerves = swerves
        

    def make_system_pose(self, pose:Pose):
        return SystemPose(pose, [Pose(pose.t, sw.offset, 0) for sw in self.swerves])


    def __str__(self):
        return f'system {self.mass:.1f}kg  {self.moment:.1f}gcm^2  {len(self.swerves)} swerves'

In [ ]:
class Power:
    def __init__(self, linear, yaw):
        asum = abs(linear) + abs(yaw)
        if asum > 1:
            linear /= asum
            yaw /= asum

        self.linear = linear
        self.yaw = yaw


    def __str__(self):
        return f'linear power {self.linear:.2%}, yaw power {self.yaw:.2%}'

In [ ]:
def project(onto, x):
    return (x @ onto) / (onto @ onto) * onto


def torque_cross(arm, f):
    return arm[0] * f[1] - arm[1] * f[0]

In [ ]:
def cross_vel(pose:Pose, vel:Velocity):
    return np.array([
        -vel.omega * pose.s[1],
        vel.omega * pose.s[0]
    ])


def to_swerve_rel(pose:Pose, vel:Velocity):
    return rot(-pose.theta) @ (vel.v + cross_vel(pose, vel))


def to_sys_rel(sys_pose:SystemPose, v:Velocity):
    return Velocity(rot(-sys_pose.pose.theta) @ v.v, v.omega)


def to_sys_rel_force(sw:Pose, sw_rel_force:np.ndarray):
    torque = np.cross(sw.s, sw_rel_force)
    axial = project(sw_rel_force, sw.s)
    return axial, torque

In [ ]:
def make_rot_vel(omega):
    return Velocity([0, 0], omega)

In [ ]:
def plot(sys, pose_hist):
    fig = plt.figure(figsize=(8, 8))

    for i in range(len(sys.swerves)):
        t, x, y = [], [], []
        for pose in pose_hist:
            s = pose.to_abs(i)
            t.append(pose.pose.t)
            x.append(s[0])
            y.append(s[1])

        plt.scatter(x, y, marker='o', c=t)

    fig.gca().set_aspect('equal') 
    plt.show()

In [ ]:
def sim(sys, sys_pose, sys_vel, max_t, get_power):
    dt, cycle_time = 1 / 200, 1 / 30
    t, next_cycle = 0, 0

    pose_hist, vel_hist = [], []
    powers = [Power(0, 0) for _ in sys.swerves]

    while t < max_t:
        sys_rel_vel = to_sys_rel(sys_pose, sys_vel)

        if verbose:
            print(f'{sys_pose} -> {sys_vel}')
            print(f'    sys_rel_vel = {sys_rel_vel}')

        yaw_list = []

        if t >= next_cycle:
            powers = get_power(sys_pose)
            next_cycle += cycle_time

        sys_rel_f, tau = np.zeros(2), 0
        for i in range(len(sys.swerves)):
            sw = sys.swerves[i]
            sw_pose = sys_pose.rel_sw_pose[i]
            sw_vel = to_swerve_rel(sw_pose, sys_rel_vel)

            if verbose:
                print(f'    swerve{i}: {sw_pose} (sys rel)')

            p = powers[i]

            yaw_list.append(make_rot_vel(sw.max_yaw_rate * p.yaw))

            applied_force = np.array([sw.max_force * p.linear, 0])
            sw_force = applied_force - sw.drag * sw_vel - sw.fstatic * np.sign(sw_vel)

            sw_f =  rot(sw_pose.theta) @ sw_force
            sw_tau = torque_cross(sw_pose.s, sw_force)

            if verbose:
                print(f'    f = {sw_f.round(2)} tau = {sw_tau:.2f}')

            sys_rel_f += sw_f
            tau += sw_tau

        sys_f = rot(sys_pose.pose.theta) @ sys_rel_f
        sys_a = Accel(sys_f / sys.mass, tau / sys.moment)

        if verbose:
            print(f'    f = {sys_f}  {sys_a}')

        sys_pose = sys_pose.advance(sys_vel, yaw_list, dt)
        sys_vel = sys_vel.advance(sys_a, dt)

        pose_hist.append(sys_pose)
        vel_hist.append(sys_vel)

        t += dt

    return pose_hist, vel_hist

In [ ]:
m0 = SwerveModule(np.array([0, 2]), np.array([0.1, 80]), 230, 1.5, 70)
m1 = SwerveModule(np.array([0, -2]), np.array([0.1, 80]), 230, 1.5, 70)
sys = System(1, 2, [m0, m1])

sys_pose = sys.make_system_pose(Pose(0, [1, 3], 10))
sys_vel = Velocity([0, 0], 0)

def get_power(pose):
    t = pose.pose.t
    if 2 < t < 3:
        return [Power(0, 1), Power(0, 1)]
    else:
        return [Power(1, 0), Power(1, 0)]
    
verbose = False
pose_hist, vel_hist = sim(sys, sys_pose, sys_vel, 6, get_power)
plot(sys, pose_hist)